In [1]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
from comtrade import Comtrade
plt.rcParams['savefig.directory'] = os.getcwd()

# change this to match the exported COMTRADE file
root = '../PSCAD/COMTRADEtest/playback'

# load all the analog channels into a dictionary of numpy arrays
rec = Comtrade ()
rec.load (root + '.cfg', root + '.dat')
t = np.array(rec.time)
channels = {}
for i in range(rec.analog_count):
    lbl = rec.analog_channel_ids[i]
    channels[lbl] = np.array (rec.analog[i])


In [2]:
print ('Channel           Min        Max       Mean')
for key, val in channels.items():
    print ('{:10s} {:10.4f} {:10.4f} {:10.4f}'.format (key, np.min(val), np.max(val), np.mean(val)))

Channel           Min        Max       Mean
 A1: VA     -113.8670   113.8812    -0.0015
 A2: VB     -114.4510   113.8711    -0.0001
 A3: VC     -119.1830   119.9054     0.0021
 A4: IA       -1.2905     1.1776    -0.0000
 A5: IB       -1.1686     1.0325     0.0004
 A6: IC       -1.3620     1.4817    -0.0004


In [ ]:
# display the channels in groups of three
%matplotlib notebook
labels = [lbl for lbl in channels]
fig, ax = plt.subplots(2, 1, sharex = 'col', figsize=(12,8), constrained_layout=True)
fig.suptitle ('Case: ' + root)
for i in range(2):
    for j in range(3):
        lbl = labels[3*i + j]
        ax[i].plot(t, channels[lbl], label=lbl)
    ax[i].grid()
    ax[i].legend()

plt.show()